# **Lab 6: Data Sources**

Lecturer: `Sirasit Lochanachit`


Course: `06026213 Big Data Systems`

Term: `02/2024`

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.master("local").getOrCreate()

In [5]:
spark

# 1. Semi-Structured Data (JSON)

## 1.1 Loading Data

ให้นักศึกษา download file แล้วบันทึกลงบน My Drive
> [arXiv Dataset](https://www.kaggle.com/datasets/Cornell-University/arxiv)

In [1]:
# ตั้งค่าคีย์ API
import os
os.makedirs('/root/.kaggle', exist_ok=True)

# ดาวน์โหลด dataset
!kaggle datasets download -d Cornell-University/arxiv

# แตกไฟล์ zip
!unzip arxiv.zip -d /content/

Dataset URL: https://www.kaggle.com/datasets/Cornell-University/arxiv
License(s): CC0-1.0
 99% 1.38G/1.39G [00:19<00:00, 121MB/s]
100% 1.39G/1.39G [00:19<00:00, 77.8MB/s]
Archive:  arxiv.zip
  inflating: /content/arxiv-metadata-oai-snapshot.json  


In [6]:
df = spark.read.json("/content/arxiv-metadata-oai-snapshot.json")
df.printSchema()

root
 |-- abstract: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- authors_parsed: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- categories: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- id: string (nullable = true)
 |-- journal-ref: string (nullable = true)
 |-- license: string (nullable = true)
 |-- report-no: string (nullable = true)
 |-- submitter: string (nullable = true)
 |-- title: string (nullable = true)
 |-- update_date: string (nullable = true)
 |-- versions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- created: string (nullable = true)
 |    |    |-- version: string (nullable = true)



In [7]:
df.show(5)

+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+------------------+--------------------+-----------+--------------------+
|            abstract|             authors|      authors_parsed|     categories|            comments|                 doi|       id|         journal-ref|             license|       report-no|         submitter|               title|update_date|            versions|
+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+------------------+--------------------+-----------+--------------------+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|         hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001|Phys.Rev.D76:0130...|                NULL|ANL-HEP-PR-07-12|    Pavel Nado

In [8]:
df.select(df['authors_parsed']).show(5, truncate=False)

+--------------------------------------------------------------------------+
|authors_parsed                                                            |
+--------------------------------------------------------------------------+
|[[Balázs, C., ], [Berger, E. L., ], [Nadolsky, P. M., ], [Yuan, C. -P., ]]|
|[[Streinu, Ileana, ], [Theran, Louis, ]]                                  |
|[[Pan, Hongjun, ]]                                                        |
|[[Callan, David, ]]                                                       |
|[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]                         |
+--------------------------------------------------------------------------+
only showing top 5 rows



In [9]:
df.count()

2661459

ดูจำนวน Partition ที่ Spark ได้ทำการแบ่งไว้

In [10]:
df.rdd.getNumPartitions()

34

สามารถกำหนดค่า Partition ใหม่ได้

In [11]:
new_df = df.repartition(30)

In [12]:
new_df.rdd.getNumPartitions()

30

เพิ่ม column ข้อมูลหมายเลข partition ของแต่ละแถว

In [13]:
from pyspark.sql.functions import spark_partition_id

In [14]:
new_df.withColumn("partitionId", spark_partition_id()).groupBy("partitionId").count().show(n=35)

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|88714|
|          1|88713|
|          2|88713|
|          3|88714|
|          4|88716|
|          5|88715|
|          6|88715|
|          7|88713|
|          8|88712|
|          9|88715|
|         10|88719|
|         11|88719|
|         12|88718|
|         13|88717|
|         14|88717|
|         15|88716|
|         16|88716|
|         17|88716|
|         18|88716|
|         19|88716|
|         20|88715|
|         21|88718|
|         22|88716|
|         23|88714|
|         24|88714|
|         25|88714|
|         26|88714|
|         27|88714|
|         28|88714|
|         29|88716|
+-----------+-----+



In [15]:
df.withColumn("partitionId", spark_partition_id()).sample(False, 0.5, 123).show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------------------+-----------+
|            abstract|             authors|      authors_parsed|          categories|            comments|                 doi|       id|         journal-ref|             license|           report-no|         submitter|               title|update_date|            versions|partitionId|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------------------+-----------+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|              hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001|Phys.

## 1.2 Dealing with missing values

Drop แถวที่มีค่า null

In [16]:
df.dropna(subset=['comments']).show(7)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+-----------------+--------------------+-----------+--------------------+
|            abstract|             authors|      authors_parsed|       categories|            comments|                 doi|       id|         journal-ref|             license|           report-no|        submitter|               title|update_date|            versions|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+-----------------+--------------------+-----------+--------------------+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|           hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001|Phys.Rev.D76:0130...|                NULL|    ANL-HEP-PR

หรือ

In [17]:
df.na.drop(subset=['comments']).show(7)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+-----------------+--------------------+-----------+--------------------+
|            abstract|             authors|      authors_parsed|       categories|            comments|                 doi|       id|         journal-ref|             license|           report-no|        submitter|               title|update_date|            versions|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+-----------------+--------------------+-----------+--------------------+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|           hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001|Phys.Rev.D76:0130...|                NULL|    ANL-HEP-PR

เติมข้อมูลที่มีค่า null

In [18]:
df.fillna(value = "unknown", subset = ['license']).show(7)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+------------------+--------------------+-----------+--------------------+
|            abstract|             authors|      authors_parsed|       categories|            comments|                 doi|       id|         journal-ref|             license|       report-no|         submitter|               title|update_date|            versions|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+------------------+--------------------+-----------+--------------------+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|           hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001|Phys.Rev.D76:0130...|             unknown|ANL-HEP-PR-07-12|    Pa

หรือ

In [19]:
df.fillna({"license": "unknown"}).show(7)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+------------------+--------------------+-----------+--------------------+
|            abstract|             authors|      authors_parsed|       categories|            comments|                 doi|       id|         journal-ref|             license|       report-no|         submitter|               title|update_date|            versions|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+------------------+--------------------+-----------+--------------------+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|           hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001|Phys.Rev.D76:0130...|             unknown|ANL-HEP-PR-07-12|    Pa

In [20]:
df = df.fillna(value = "unknown", subset = ['license'])
df = df.na.drop(subset=['comments'])

In [21]:
df.count()

1978671

## 1.3 Data Exploration

### A. ดึงข้อมูล authors ที่ตีพิมพ์บทความวิชาการในหมวดหมู่ของ math

แบบใช้ filter() หรือ where()

In [22]:
df.filter(df['categories'].like('%math%')).select('authors', 'categories').show(10)

+--------------------+---------------+
|             authors|     categories|
+--------------------+---------------+
|Ileana Streinu an...|  math.CO cs.CG|
|        David Callan|        math.CO|
|  Sergei Ovchinnikov|        math.CO|
|Clifton Cunningha...|math.NT math.AG|
|        Koichi Fujii|math.CA math.AT|
|         Norio Konno|math.PR math.AG|
|Simon J.A. Malham...|        math.NA|
|Robert P. C. de M...|        math.RA|
|  P\'eter E. Frenkel|math.CA math.PR|
|          Mihai Popa|math.OA math.FA|
+--------------------+---------------+
only showing top 10 rows



In [23]:
df.filter(df['categories'].like('%math%')).count()

465746

แบบ pandas

In [24]:
df[df['categories'].like('%math%')].count()

465746

แบบระบุ condition เป็น string

In [25]:
df.where("categories LIKE '%math%'").count()

465746

แบบ Spark SQL

In [26]:
df.createOrReplaceTempView("arXiv")

In [27]:
sql_query = """
SELECT authors, categories
FROM arXiv
WHERE categories LIKE '%math%'
"""

In [28]:
spark.sql(sql_query).show(10)

+--------------------+---------------+
|             authors|     categories|
+--------------------+---------------+
|Ileana Streinu an...|  math.CO cs.CG|
|        David Callan|        math.CO|
|  Sergei Ovchinnikov|        math.CO|
|Clifton Cunningha...|math.NT math.AG|
|        Koichi Fujii|math.CA math.AT|
|         Norio Konno|math.PR math.AG|
|Simon J.A. Malham...|        math.NA|
|Robert P. C. de M...|        math.RA|
|  P\'eter E. Frenkel|math.CA math.PR|
|          Mihai Popa|math.OA math.FA|
+--------------------+---------------+
only showing top 10 rows



### [Exercise 6-1] B. ดึงข้อมูล license ที่มี abstract ความยาว 5 ตัวอักษรขึ้นไป

In [29]:
from pyspark.sql.functions import length

แบบใช้ filter() หรือ where()

In [ ]:
df.filter(length(df['abstract']) > 5).select('license', 'abstract').show(10), df.filter(length(df['abstract']) > 5).count()

+--------------------+--------------------+
|             license|            abstract|
+--------------------+--------------------+
|             unknown|  A fully differe...|
|http://arxiv.org/...|  We describe a n...|
|             unknown|  The evolution o...|
|             unknown|  We show that a ...|
|             unknown|  We study the tw...|
|             unknown|  A rather non-st...|
|http://arxiv.org/...|  A general formu...|
|             unknown|  Partial cubes a...|
|http://arxiv.org/...|  In this paper w...|
|             unknown|  In this article...|
+--------------------+--------------------+
only showing top 10 rows



(None, 1978671)

แบบ pandas

In [36]:
df[length(df['abstract']) > 5]['license', 'abstract'].show(10), df[length(df['abstract']) > 5]['license', 'abstract'].count()

+--------------------+--------------------+
|             license|            abstract|
+--------------------+--------------------+
|             unknown|  A fully differe...|
|http://arxiv.org/...|  We describe a n...|
|             unknown|  The evolution o...|
|             unknown|  We show that a ...|
|             unknown|  We study the tw...|
|             unknown|  A rather non-st...|
|http://arxiv.org/...|  A general formu...|
|             unknown|  Partial cubes a...|
|http://arxiv.org/...|  In this paper w...|
|             unknown|  In this article...|
+--------------------+--------------------+
only showing top 10 rows



(None, 1978671)

แบบระบุ condition เป็น string

In [ ]:
df.where("length(abstract) > 5").select('license', 'abstract').show(10), df.where("length(abstract) > 5").count()

+--------------------+--------------------+
|             license|            abstract|
+--------------------+--------------------+
|             unknown|  A fully differe...|
|http://arxiv.org/...|  We describe a n...|
|             unknown|  The evolution o...|
|             unknown|  We show that a ...|
|             unknown|  We study the tw...|
|             unknown|  A rather non-st...|
|http://arxiv.org/...|  A general formu...|
|             unknown|  Partial cubes a...|
|http://arxiv.org/...|  In this paper w...|
|             unknown|  In this article...|
+--------------------+--------------------+
only showing top 10 rows



(None, 1978671)

แบบ Spark SQL

In [37]:
# df.createOrReplaceTempView("arXiv")
sql_query = """
SELECT license, abstract
FROM arXiv
WHERE length(abstract) > 5
"""

In [38]:
spark.sql(sql_query).show(10), spark.sql(sql_query).count()

+--------------------+--------------------+
|             license|            abstract|
+--------------------+--------------------+
|             unknown|  A fully differe...|
|http://arxiv.org/...|  We describe a n...|
|             unknown|  The evolution o...|
|             unknown|  We show that a ...|
|             unknown|  We study the tw...|
|             unknown|  A rather non-st...|
|http://arxiv.org/...|  A general formu...|
|             unknown|  Partial cubes a...|
|http://arxiv.org/...|  In this paper w...|
|             unknown|  In this article...|
+--------------------+--------------------+
only showing top 10 rows



(None, 1978671)

### C. แสดงค่าเฉลี่ยของจำนวนหน้าสำหรับแต่ละ category

In [ ]:
from pyspark.sql.functions import split, explode, avg

df_with_pages = df.withColumn("pages", split(df["comments"], " ")[0].cast("int"))
average_pages_by_category = df_with_pages.groupBy("categories").agg(avg("pages").alias("avg_pages"))
average_pages_by_category.show()


In [ ]:
average_pages_by_category.count()

### [Exercise 6-2] D. แสดงค่าเฉลี่ยของจำนวนหน้าสำหรับ license ที่เป็น unknown

In [ ]:
from pyspark.sql.functions import when, split, explode, avg

# comments = '37 pages, 15 figu...'
df_with_pages = df.withColumn("pages", when(split(df["comments"], " ")[0].cast("int").isNotNull(), split(df["comments"], " ")[0].cast("int")).otherwise(0))
# isNotNull() = if -> 37, otherwise() = else -> 0 ==> keep it in pages column
df_with_pages.show(5)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------+--------------------+-----------+--------------------+-----+
|            abstract|             authors|      authors_parsed|       categories|            comments|                 doi|       id|         journal-ref|             license|       report-no|     submitter|               title|update_date|            versions|pages|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------+--------------------+-----------+--------------------+-----+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|           hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001|Phys.Rev.D76:0130...|             unknown|ANL-HEP-PR-07-12|

In [ ]:
unknown_license_df = df_with_pages.filter(df_with_pages["license"] == "unknown")
average_pages_by_category_unknown = unknown_license_df.groupBy("categories").agg(avg("pages").alias("avg_pages"))

average_pages_by_category_unknown.show()

+--------------------+------------------+
|          categories|         avg_pages|
+--------------------+------------------+
|     math.DG math.RT| 19.73170731707317|
|physics.gen-ph ph...|10.594202898550725|
|     math.DG math.DS|14.482758620689655|
|     math.MG math.DS|               7.0|
|nlin.CD physics.s...|              24.0|
|math.CO math.AT m...|              25.0|
|stat.ME stat.AP s...|               0.0|
|astro-ph gr-qc ph...|13.666666666666666|
|cond-mat.dis-nn c...|               2.0|
|cond-mat.str-el p...|10.666666666666666|
|cond-mat.soft con...|              12.4|
|physics.class-ph ...|               7.0|
|gr-qc astro-ph qu...|              14.3|
|     math.AP math.CA|             14.75|
|math.PR math.ST s...|12.991071428571429|
|             nlin.CD| 10.90119971771348|
|     math.GR math.GT|15.988826815642458|
|         cs.RO cs.NE|              16.0|
|quant-ph gr-qc he...|11.920353982300885|
|     math.PR math.NA| 13.88888888888889|
+--------------------+------------

In [ ]:
average_pages_by_category_unknown.count()

12883

---

# 2. Semi-Structured/Unstructured Data (log file)

## 2.1 Loading Data

ให้นักศึกษา download file แล้วบันทึกลงบน My Drive
> [GHtorrent Data](https://drive.google.com/file/d/1pKtPpNvpJ8nSqGfnpXsd5fGsFvk6a_E0/view?usp=sharing)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
rdd = spark.sparkContext.textFile('/content/drive/MyDrive/bigDataSystems/ghtorrent-logs.txt.gz')

In [ ]:
rdd.getNumPartitions()

1

In [ ]:
spark.sparkContext.defaultParallelism

1

In [ ]:
# Repartition
rdd = rdd.repartition(8) # shuffle all data
print(spark.sparkContext.defaultParallelism)
print(rdd.getNumPartitions())

1
8


In [ ]:
# Cache Data
from pyspark import StorageLevel
rdd.persist(StorageLevel.MEMORY_AND_DISK)

MapPartitionsRDD[147] at coalesce at NativeMethodAccessorImpl.java:0

In [ ]:
rdd.take(5)

['INFO, 2017-03-23T13:00:55+00:00, ghtorrent-42 -- api_client.rb: Successful request. URL: https://api.github.com/repos/CanonicalLtd/maas-docs/issues/365/events?per_page=100, Remaining: 4943, Total: 88 ms',
 'WARN, 2017-03-23T20:04:28+00:00, ghtorrent-13 -- api_client.rb: Failed request. URL: https://api.github.com/repos/greatfakeman/Tabchi/commits?sha=Tabchi&per_page=100, Status code: 404, Status: Not Found, Access: ac6168f8776, IP: 0.0.0.0, Remaining: 3031',
 'DEBUG, 2017-03-23T09:06:09+00:00, ghtorrent-2 -- ghtorrent.rb: Transaction committed (11 ms)',
 'DEBUG, 2017-03-23T09:08:59+00:00, ghtorrent-45 -- ghtorrent.rb: Repo vert-x3/vertx-lang-scala exists',
 'DEBUG, 2017-03-23T12:11:59+00:00, ghtorrent-1 -- ght_data_retrieval.rb: Processing event: PullRequestEvent-5532641148']

In [ ]:
rdd.count()

9669788

สุ่มข้อมูล

In [ ]:
rdd.takeSample(False, 20, 1234) # withReplacement, size, random seed

['INFO, 2017-03-23T09:21:05+00:00, ghtorrent-40 -- api_client.rb: Successful request. URL: https://api.github.com/repos/ssor/wu/forks?page=1&per_page=100, Remaining: 1548, Total: 71 ms',
 'DEBUG, 2017-03-23T09:04:46+00:00, ghtorrent-11 -- ghtorrent.rb: Repo vodaka/forum exists',
 'DEBUG, 2017-03-24T12:04:11+00:00, ghtorrent-48 -- retriever.rb: Commit iam1980/PoshC2 -> 0b6b34645823595d3e7dab4778d944dc8a29e4fe exists',
 'DEBUG, 2017-03-23T09:06:58+00:00, ghtorrent-2 -- ghtorrent.rb: Repo kalefranz/conda exists',
 'DEBUG, 2017-03-23T10:10:41+00:00, ghtorrent-27 -- ghtorrent.rb: Association of commit a7c404de25f1e5ee5f971b14ab90038caa35472f with repo CartoDB/cartodb exists',
 'INFO, 2017-03-23T09:12:25+00:00, ghtorrent-3 -- ghtorrent.rb: Added project_language coder0718/thirdparty -> M4 (248024 bytes)',
 'DEBUG, 2017-03-23T09:08:07+00:00, ghtorrent-11 -- ghtorrent.rb: Repo ResurrectionRemix/android_frameworks_base exists',
 'DEBUG, 2017-03-23T10:21:41+00:00, ghtorrent-41 -- ghtorrent.rb: T

### GHTorrent data format
ในแต่ละบรรทัดของ log file ประกอบด้วย

1. Logging level: DEBUG, INFO, WARN, ERROR
2. timestamp
3. downloader id
4. logging stage ประกอบด้วยอย่างน้อย 1 อย่าง
  - event_processing
  - ght_data_retrieval
  - api_client
  - retriever
  - ghtorrent

## 2.2 Data Exploration

### A. ดึงข้อมูลจำนวนบรรทัดที่มีคำว่า transaction หรือ repo

In [ ]:
from pyspark.sql.functions import col

DataFrame version

In [ ]:
df_logs = spark.createDataFrame(rdd, "string").toDF("line")

In [ ]:
df_logs.show(5, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|line                                                                                                                                                                                                                                                         |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|INFO, 2017-03-23T13:00:55+00:00, ghtorrent-42 -- api_client.rb: Successful request. URL: https://api.github.com/repos/CanonicalLtd/maas-docs/issues/365/events?per_page=100, Remaining: 4943, Total: 88 ms                             

In [ ]:
# Get the number of lines with both Transaction or Repo information
# Split each line into columns
split_col = split(df_logs["line"], " ")
df_logs = df_logs.withColumn("level", split_col.getItem(0)) \
    .withColumn("timestamp", split_col.getItem(1)) \
    .withColumn("downloader", split_col.getItem(2))

df_logs.show(5)

+--------------------+------+--------------------+------------+
|                line| level|           timestamp|  downloader|
+--------------------+------+--------------------+------------+
|INFO, 2017-03-23T...| INFO,|2017-03-23T13:00:...|ghtorrent-42|
|WARN, 2017-03-23T...| WARN,|2017-03-23T20:04:...|ghtorrent-13|
|DEBUG, 2017-03-23...|DEBUG,|2017-03-23T09:06:...| ghtorrent-2|
|DEBUG, 2017-03-23...|DEBUG,|2017-03-23T09:08:...|ghtorrent-45|
|DEBUG, 2017-03-23...|DEBUG,|2017-03-23T12:11:...| ghtorrent-1|
+--------------------+------+--------------------+------------+
only showing top 5 rows



In [ ]:
transaction_repo_lines = df_logs.filter((col("line").contains("transaction")) | (col("line").contains("repo")))
# transaction_repo_lines = df_logs.filter((col("line").like("%transaction%")) | (col("line").like("%repo%")))
num_lines = transaction_repo_lines.count()

In [ ]:
print(f"Number of lines with both Transaction or Repo: {num_lines}")

Number of lines with both Transaction or Repo: 1433659


RDD Version

In [ ]:
transaction_repo_lines_rdd = rdd.filter(lambda line: "transaction" in line or "repo" in line)
num_lines_rdd = transaction_repo_lines_rdd.count()

In [ ]:
print(f"Number of lines with Transaction or Repo (RDD): {num_lines_rdd}")


Number of lines with Transaction or Repo (RDD): 1433659


### [Exercise 6-3] B. ดึงข้อมูลจำนวนบรรทัดที่มี URL Link สำหรับ Logging Level ที่เป็น WARN

DataFrame Version

In [ ]:
warn_lines_with_urls = df_logs.filter((col("level").contains("WARN")) & (col("line").contains("http")))
num_warn_lines_with_urls = warn_lines_with_urls.count()

In [ ]:
print(f"Number of WARN lines with URLs:  {num_warn_lines_with_urls}")

Number of WARN lines with URLs:  95346


RDD Version

In [ ]:
warn_lines_with_urls_rdd = rdd.filter(lambda line: "WARN" in line and "http" in line)
num_warn_lines_with_urls_rdd = warn_lines_with_urls_rdd.count()

In [ ]:
print(f"Number of WARN lines with URLs: {num_warn_lines_with_urls_rdd}")

Number of WARN lines with URLs: 95346


### [Exercise 6-4] C. Downloader id ใดที่มี Failed connection มากที่สุด

In [ ]:
df_logs = df_logs.withColumn("connection_status", split_col.getItem(5))
df_logs.show(10)

+--------------------+------+--------------------+------------+-----------------+
|                line| level|           timestamp|  downloader|connection_status|
+--------------------+------+--------------------+------------+-----------------+
|INFO, 2017-03-23T...| INFO,|2017-03-23T13:00:...|ghtorrent-42|       Successful|
|WARN, 2017-03-23T...| WARN,|2017-03-23T20:04:...|ghtorrent-13|           Failed|
|DEBUG, 2017-03-23...|DEBUG,|2017-03-23T09:06:...| ghtorrent-2|      Transaction|
|DEBUG, 2017-03-23...|DEBUG,|2017-03-23T09:08:...|ghtorrent-45|             Repo|
|DEBUG, 2017-03-23...|DEBUG,|2017-03-23T12:11:...| ghtorrent-1|       Processing|
|DEBUG, 2017-03-23...|DEBUG,|2017-03-23T12:06:...|ghtorrent-44|           Commit|
|DEBUG, 2017-03-23...|DEBUG,|2017-03-23T11:03:...|ghtorrent-17|             User|
|DEBUG, 2017-03-23...|DEBUG,|2017-03-23T11:16:...| ghtorrent-8|           Commit|
|DEBUG, 2017-03-23...|DEBUG,|2017-03-23T09:23:...|ghtorrent-33|             Repo|
|DEBUG, 2017-03-

In [ ]:
failed_connections = df_logs.filter(col("connection_status") == "Failed")
# failed_connections = df_logs.filter(col("connection_status").contains("Failed"))
downloader_counts = failed_connections.groupBy("downloader").count()
downloader_counts.show(5)

+------------+-----+
|  downloader|count|
+------------+-----+
|ghtorrent-45|  310|
|ghtorrent-17|  263|
|ghtorrent-49|  202|
|ghtorrent-15|  262|
|ghtorrent-38|  247|
+------------+-----+
only showing top 5 rows



In [ ]:
most_active_downloader = downloader_counts.orderBy(col("count").desc()).first()
most_active_downloader

Row(downloader='ghtorrent-13', count=79623)

In [ ]:
if most_active_downloader:
    print(f"The most active downloader ID for 'Failed' connections is: {most_active_downloader['downloader']} with {most_active_downloader['count']} failed connections.")
else:
    print("No 'Failed' connections found.")

The most active downloader ID for 'Failed' connections is: ghtorrent-13 with 79623 failed connections.


The most active downloader ID for 'Failed' connections is: ghtorrent-13 with 18537 failed connections.


### D. คำนวณค่าเฉลี่ยของความยาวตัวอักษรสำหรับแต่ละบรรทัด

RDD Version

In [ ]:
rdd.map(lambda line: len(line)).mean()

124.73147156897309

In [ ]:
length_rdd = rdd.map(lambda line: len(line))
summation_rdd = length_rdd.reduce(lambda x, y: x + y)
average_length_rdd = summation_rdd / length_rdd.count()
average_length_rdd

124.73147156897339

DataFrame Version

In [ ]:
from pyspark.sql.functions import length

In [ ]:
df_logs.select(length(col('line'))).show()

+------------+
|length(line)|
+------------+
|         202|
|         253|
|          92|
|         100|
|         117|
|         167|
|          85|
|         153|
|          89|
|         113|
|         100|
|         141|
|         113|
|         148|
|          91|
|          92|
|         190|
|         134|
|         108|
|          96|
+------------+
only showing top 20 rows



In [ ]:
df_logs.select(length(col('line'))).groupBy().avg().show()

+------------------+
| avg(length(line))|
+------------------+
|124.73147156897339|
+------------------+



In [ ]:
df_logs.select(length(col('line'))).groupBy().avg().collect()[0][0]

124.73147156897339

### [Exercise 6-5] E. ค้นหา Log entry ที่ยาวที่สุด

RDD Version

In [ ]:
rdd.max(key=lambda line: len(line))

"E, [2017-03-22T20:22:54.022823 #36566] ERROR -- #<Bunny::Session:30302160 ghtorrent@streamer:5672, vhost=/, hosts=[streamer]>: Uncaught exception from consumer #<Bunny::Consumer:70104670867560 @channel_id=1 @queue=PullRequestReviewCommentEvents> @consumer_tag=bunny-1490213459000-837038195849>: #<NoMethodError: undefined method `[]' for nil:NilClass> @ /home/gousiosg/github-mirror/lib/ghtorrent/commands/ght_data_retrieval.rb:104:in `rescue in block (2 levels) in go'"

DataFrame Version

In [ ]:
df_logs.select(length(col('line'))).orderBy(length(col('line')).desc()).show(1)

+------------+
|length(line)|
+------------+
|         469|
+------------+
only showing top 1 row



---